In [2]:
import gymnasium as gym
import torch as t
import gymnasium as gym

from stable_baselines3 import DQN, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement, CheckpointCallback


## Simple Reinforcement Learning

Simple test loop with lunar lander to test random action (without model)
and see if reward and action flow is working as expected

In [1]:
import gymnasium as gym

env = gym.make("LunarLander-v2", render_mode="human")

def get_action(env, model = None, obs = None):
    action = 0
    _states = None

    if model is None:
        action = env.action_space.sample()  # agent policy that uses the observation and info
        print(f"Action taken is: {action}")
    elif model is not None:
        action, _states = model.predict(obs, deterministic=True)
    else:
        action = 0
    return action, _states


def run_environment(env, model = None, verbose: bool = False):

    observation, info = env.reset()
    for _ in range(1000):

        action, _states = get_action(env=env, model=model, obs=observation)
        observation, reward, terminated, truncated, info = env.step(action)

        print(f"Current Reward: {reward}")

        if terminated or truncated:
            print("Game is Over!")
            observation, info = env.reset()

    env.close()

In [2]:
run_environment(env=env, model=None, verbose=True)

Action taken is: 3
Current Reward: 0.5336336294087698
Action taken is: 2
Current Reward: -1.3591815551159414
Action taken is: 0
Current Reward: -0.5169990319246267
Action taken is: 3
Current Reward: 0.5100320852958771
Action taken is: 0
Current Reward: -0.38620506834004686
Action taken is: 3
Current Reward: 0.5342914878027056
Action taken is: 3
Current Reward: 0.6771096892316859
Action taken is: 3
Current Reward: 1.1522431266751323
Action taken is: 1
Current Reward: -1.117113815819066
Action taken is: 3
Current Reward: 0.7438421490318501
Action taken is: 0
Current Reward: -0.07078228232546735
Action taken is: 0
Current Reward: -0.1119046402337176
Action taken is: 0
Current Reward: -0.15250740704021837
Action taken is: 2
Current Reward: -1.1690165987281944
Action taken is: 1
Current Reward: -1.33270106775464
Action taken is: 3
Current Reward: 0.9161701611658384
Action taken is: 2
Current Reward: 0.7766691928721456
Action taken is: 3
Current Reward: 0.8258352117125025
Action taken is: 2


### View sample observation/action space and observation space

In [3]:
env = gym.make("LunarLander-v2", render_mode="human")
obs = env.reset()
env.close()

In [4]:
obs

(array([-0.00712576,  1.4217155 , -0.7217721 ,  0.47977316,  0.00826371,
         0.16349222,  0.        ,  0.        ], dtype=float32),
 {})

In [5]:
env.action_space

Discrete(4)

In [6]:
env.observation_space

Box([-1.5       -1.5       -5.        -5.        -3.1415927 -5.
 -0.        -0.       ], [1.5       1.5       5.        5.        3.1415927 5.        1.
 1.       ], (8,), float32)

## Model training

In [7]:
import torch as t

In [8]:
t.cuda.is_available()

True

In [9]:
%tensorboard --logdir lunar_lander_logs/

UsageError: Line magic function `%tensorboard` not found.


In [10]:
import gymnasium as gym

from stable_baselines3 import DQN, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement, CheckpointCallback



# Create environment
env = gym.make("LunarLander-v2", render_mode="rgb_array")


# Stop Training callback
stop_train_callback = StopTrainingOnNoModelImprovement(
    max_no_improvement_evals=3_000, 
    min_evals=500, 
    verbose=1
)
eval_callback = EvalCallback(
    env,
    best_model_save_path="./lunar_lander_logs/",
    log_path="./lunar_lander_logs/",
    eval_freq=1_000, 
    callback_after_eval=stop_train_callback, 
    verbose=1,
    deterministic=True, 
    render=False,

)


# Instantiate the agent
model = PPO(
    "MlpPolicy", 
    env, 
    verbose=1, 
    device="cuda", 
    tensorboard_log="./lunar_lander_logs/",
    ent_coef=0.01
)
# Train the agent and display a progress bar
model.learn(
    total_timesteps=int(300_000), 
    progress_bar=True,
    callback=eval_callback,
)
# Save the agent
model.save("dqn_lunar")
del model  # delete trained model to demonstrate loading


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./logs/PPO_9


/home/copito/Documents/simple_rl/venv/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/copito/Documents/simple_rl/venv/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: 
UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting 
modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first
with ``Monitor`` wrapper.
  warnings.warn(

Eval num_timesteps=1000, episode_reward=-382.11 +/- 61.88

Episode length: 64.00 +/- 11.52

...

Episode length: 238.00 +/- 17.57

------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 238          |
|    mean_reward          | 226          |
| time/                   |              |
|    total_timesteps      | 300000       |
| train/                  |              |
|    approx_kl            | 0.0039451467 |
|    clip_fraction        | 0.035        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.615       |
|    explained_variance   | 0.777        |
|    learning_rate        | 0.0003       |
|    loss                 | 46.1         |
|    n_updates            | 1460         |
|    policy_gradient_loss | -0.00163     |
|    value_loss           | 128          |
------------------------------------------


Eval num_timesteps=301000, episode_reward=264.35 +/- 10.96

Episode length: 264.40 +/- 27.99

---------------------------------
| eval/              |          |
|    mean_ep_length  | 264      |
|    mean_reward     | 264      |
| time/              |          |
|    total_timesteps | 301000   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 322      |
|    ep_rew_mean     | 254      |
| time/              |          |
|    fps             | 143      |
|    iterations      | 147      |
|    time_elapsed    | 2096     |
|    total_timesteps | 301056   |
---------------------------------


In [4]:

# Load the trained agent
# NOTE: if you have loading issue, you can pass `print_system_info=True`
# to compare the system on which the model was trained vs the current one
# model = DQN.load("dqn_lunar", env=env, print_system_info=True)
env = gym.make("LunarLander-v2", render_mode="rgb_array")
model = PPO.load("./lunar_lander_logs/best_model.zip", env=env)

# Evaluate the agent
# NOTE: If you use wrappers with your environment that modify rewards,
#       this will be reflected here. To evaluate with original rewards,
#       wrap environment in a "Monitor" wrapper before other wrappers.
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
mean_reward

260.5610057

In [6]:
# Enjoy trained agent
total_reward = 0
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(10_000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    total_reward += rewards

    vec_env.render("human")

print(f"Reward: {total_reward}")
vec_env.close()

Reward: [10197.532]


In [14]:
model = PPO.load('./lunar_lander_logs/best_model.zip')
model.set_env(env=env)
# Train the agent and display a progress bar
model.learn(
    total_timesteps=int(300_000), 
    progress_bar=True,
    callback=eval_callback
)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./logs/PPO_10


Eval num_timesteps=944, episode_reward=162.95 +/- 109.97

Episode length: 243.80 +/- 35.89

---------------------------------
| eval/              |          |
|    mean_ep_length  | 244      |
|    mean_reward     | 163      |
| time/              |          |
|    total_timesteps | 944      |
---------------------------------


Eval num_timesteps=1944, episode_reward=155.83 +/- 89.01

...

-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 328         |
|    mean_reward          | 249         |
| time/                   |             |
|    total_timesteps      | 299944      |
| train/                  |             |
|    approx_kl            | 0.008010677 |
|    clip_fraction        | 0.0857      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.679      |
|    explained_variance   | 0.927       |
|    learning_rate        | 0.0003      |
|    loss                 | 7.75        |
|    n_updates            | 2750        |
|    policy_gradient_loss | -0.00188    |
|    value_loss           | 14.2        |
-----------------------------------------


Eval num_timesteps=300944, episode_reward=278.93 +/- 24.62

Episode length: 234.80 +/- 25.16

---------------------------------
| eval/              |          |
|    mean_ep_length  | 235      |
|    mean_reward     | 279      |
| time/              |          |
|    total_timesteps | 300944   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 300      |
|    ep_rew_mean     | 293      |
| time/              |          |
|    fps             | 223      |
|    iterations      | 147      |
|    time_elapsed    | 1347     |
|    total_timesteps | 301056   |
---------------------------------


: 

In [20]:
model = PPO.load("./lunar_lander_logs/best_model.zip", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## Putting through the fire!

In [7]:
# Enjoy trained agent
total_reward = 0
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(10_000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    total_reward += rewards

    vec_env.render("human")

print(f"Reward: {total_reward}")
vec_env.close()

Reward: [11530.818]
